In [ ]:
# Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import newton

In [ ]:
# Fixed Rate Bond Class: Clean instrument definition (no pricing logic inside).

class FixedRateBond:
    def __init__(self, face, coupon_rate, maturity, freq=2):
        self.face = face
        self.coupon_rate = coupon_rate
        self.maturity = maturity
        self.freq = freq

    def cashflows(self):
        periods = int(self.maturity * self.freq)
        coupon = self.face * self.coupon_rate / self.freq
        flows = np.full(periods, coupon)
        flows[-1] += self.face
        return flows

    def times(self):
        periods = int(self.maturity * self.freq)
        return np.array([(i+1)/self.freq for i in range(periods)])

In [ ]:
# Yield Curve Class (Continuous Compounding): Arbitrage-consistent discounting.

class YieldCurve:
    def __init__(self, times, rates):
        self.times = np.array(times)
        self.rates = np.array(rates)

    def get_spot_rate(self, t):
        return np.interp(t, self.times, self.rates)

    def discount_factor(self, t):
        r = self.get_spot_rate(t)
        return np.exp(-r * t)

In [ ]:
# Discounting Pricing Engine

class DiscountingPricer:

    def price(self, bond, curve):
        cashflows = bond.cashflows()
        times = bond.times()

        price = 0
        for cf, t in zip(cashflows, times):
            df = curve.discount_factor(t)
            price += cf * df

        return price

In [ ]:
# Create a Sample Yield Curve (Normal Market): Smooth upward-sloping curve.

curve_times = [0.5, 1, 2, 3, 5, 7, 10]
curve_rates = [0.03, 0.032, 0.035, 0.037, 0.04, 0.042, 0.045]

yield_curve = YieldCurve(curve_times, curve_rates)

plt.plot(curve_times, curve_rates)
plt.title("Normal Market Yield Curve")
plt.xlabel("Maturity (Years)")
plt.ylabel("Spot Rate")
plt.grid(True)
plt.show()

In [ ]:
# Create a Bond

bond = FixedRateBond(face=1000, coupon_rate=0.05, maturity=5, freq=2)

In [ ]:
# Price the Bond

pricer = DiscountingPricer()
price = pricer.price(bond, yield_curve)

print(f"Bond Price: {price:.2f}")

In [ ]:
# Yield to Maturity Solver: Now solve flat YTM implied by price.

def price_with_ytm(bond, ytm):
    cashflows = bond.cashflows()
    times = bond.times()
    return sum(cf / (1 + ytm/bond.freq)**(t*bond.freq)
               for cf, t in zip(cashflows, times))

def solve_ytm(bond, market_price):
    func = lambda y: price_with_ytm(bond, y) - market_price
    return newton(func, 0.05)

In [ ]:
# Print Implied YTM

ytm = solve_ytm(bond, price)
print(f"Implied YTM: {ytm:.4%}")

In [ ]:
# Risk Analytics: DV01 (Parallel Shift)

def dv01(pricer, bond, curve, bump=0.0001):
    base_price = pricer.price(bond, curve)

    bumped_curve = YieldCurve(curve.times, curve.rates + bump)
    bumped_price = pricer.price(bond, bumped_curve)

    return bumped_price - base_price

In [ ]:
#Print DV01

risk = dv01(pricer, bond, yield_curve)
print(f"DV01: {risk:.4f}")

In [ ]:
# Modified Duration (Numerical)

def modified_duration(pricer, bond, curve, bump=0.0001):
    base_price = pricer.price(bond, curve)

    up_curve = YieldCurve(curve.times, curve.rates + bump)
    down_curve = YieldCurve(curve.times, curve.rates - bump)

    up_price = pricer.price(bond, up_curve)
    down_price = pricer.price(bond, down_curve)

    return (down_price - up_price) / (2 * base_price * bump)

In [ ]:
# Print Modified Duration

duration = modified_duration(pricer, bond, yield_curve)
print(f"Modified Duration: {duration:.4f}")

In [ ]:
# Portfolio Extension: price multiple bonds.

bonds = [
    FixedRateBond(1000, 0.04, 3),
    FixedRateBond(1000, 0.05, 5),
    FixedRateBond(1000, 0.06, 7)
]

portfolio_value = sum(pricer.price(b, yield_curve) for b in bonds)

print(f"Portfolio Value: {portfolio_value:.2f}")

In [ ]:
#Final Results Display Code (Single Summary Output)

# COMPUTE ALL RESULTS

price = pricer.price(bond, yield_curve)
ytm = solve_ytm(bond, price)
risk = dv01(pricer, bond, yield_curve)
duration = modified_duration(pricer, bond, yield_curve)

portfolio_value = sum(pricer.price(b, yield_curve) for b in bonds)

# ORGANIZE RESULTS INTO TABLE

results = {
    "Metric": [
        "Bond Price",
        "Implied YTM",
        "DV01",
        "Modified Duration",
        "Portfolio Value"
    ],
    "Value": [
        round(price, 2),
        f"{ytm:.4%}",
        round(risk, 6),
        round(duration, 4),
        round(portfolio_value, 2)
    ]
}

results_df = pd.DataFrame(results)


# DISPLAY RESULTS

print("\n==============================")
print(" FIXED INCOME ENGINE RESULTS ")
print("==============================\n")

display(results_df)

In [ ]:
# Sensitivity Visualization

shifts = np.linspace(-0.02, 0.02, 50)
prices = []

for shift in shifts:
    shifted_curve = YieldCurve(
        yield_curve.times,
        yield_curve.rates + shift
    )
    prices.append(pricer.price(bond, shifted_curve))

plt.plot(shifts * 100, prices)
plt.title("Bond Price Sensitivity to Parallel Yield Shifts")
plt.xlabel("Yield Shift (%)")
plt.ylabel("Bond Price")
plt.grid(True)
plt.show()